In [1]:
pip install selenium webdriver-manager beautifulsoup4


  Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-win_amd64.whl.metadata (34 kB)
Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl (27 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.5-py3-none-any.whl (36 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.3.2-cp312-cp312-win_amd64.whl (100 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


IMPORTING LIBRARIES

In [ ]:
pip install selenium webdriver-manager beautifulsoup4

EXTRACT THE GOV BUSES ROUTE PAGE LINK FROM REDBUS WEBSITE PAGE AND PRINT EACH STATE GOVBUS URL IN FORM OF DICTIONARY

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time


def region_links(driver,url):

    # Open the Redbus website
    driver.get(url)

    # Wait for the page to load
    time.sleep(5)

    # Extract the government buses URL
    try:
        government_buses_section = driver.find_element(By.XPATH, "//div[contains(@class, 'loaderHeading') and text()='Government Buses']/ancestor::div[contains(@class, 'rtcSection')]")
        government_buses_url = government_buses_section.find_element(By.XPATH, ".//div[contains(@class, 'rtcHeadViewAll')]//a").get_attribute("href")

        # Ensure the government buses URL is correctly formatted
        if not government_buses_url.startswith(url):
            government_buses_url = url + government_buses_url

        # Print the government buses URL
        print("Government Buses URL:", government_buses_url)

        # Open the government buses URL
        driver.get(government_buses_url)

        # Wait for the page to load
        time.sleep(5)

        # Navigate to Redbus website (update URL to the specific page you are scraping)
        # url = "https://www.redbus.in/online-booking/rtc-directory"
        # driver.get(url)

        # Wait for the page to load completely
        driver.implicitly_wait(10)  # Wait for up to 10 seconds

        # Find all elements with class "D113_ul_region_rtc"
        regions = driver.find_elements(By.CLASS_NAME, "D113_ul_region_rtc")

        # Dictionary to store region names and their corresponding href values
        region_dict = {}

        # Iterate through each region element
        for region in regions:
            # Extract region name from the <strong> tag
            region_name = region.find_element(By.TAG_NAME, 'strong').text
            
            # Extract all href tags within the region element
            href_elements = region.find_elements(By.TAG_NAME, 'a')
            
            # List to store href values
            hrefs = []
            for href in href_elements:
                href_url = href.get_attribute('href')
                # Ensure the href URL is correctly formatted
                if not href_url.startswith(url):
                    href_url = url + href_url
                hrefs.append(href_url)
            
            # Add to dictionary
            region_dict[region_name] = hrefs

    finally:
        # Close the WebDriver
        driver.quit()
    return region_dict


In [2]:
# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Define the URL of the Redbus website
url = "https://www.redbus.in"

region_dict=region_links(driver,url)

Government Buses URL: https://www.redbus.in/online-booking/rtc-directory


In [4]:
region_dict

{'South': ['https://www.redbus.in/online-booking/tnstc',
  'https://www.redbus.in/online-booking/puducherry-road-transport-corporation-prtc',
  'https://www.redbus.in/online-booking/ktcl',
  'https://www.redbus.in/online-booking/ksrtc-kerala',
  'https://www.redbus.in/online-booking/apsrtc',
  'https://www.redbus.in/online-booking/tsrtc'],
 'East': ['https://www.redbus.in/online-booking/bihar-state-tourism-development-corporation',
  'https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc',
  'https://www.redbus.in/online-booking/west-bengal-transport-corporation',
  'https://www.redbus.in/online-booking/north-bengal-state-transport-corporation',
  'https://www.redbus.in/online-booking/bsrtc-operated-by-vip-travels',
  'https://www.redbus.in/online-booking/wbtc-ctc',
  'https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc'],
 'North': ['https://www.redbus.in/online-booking/pepsu',
  'https://www.redbus.in/online-booking/rsrtc',


USING THE REGION_DICT EXTRACTING THE ROUTE LINKS FOR EACH STATE GOV BUS ROUTES

In [3]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def fetch_route_details(url):
    route_details = {}
    driver = webdriver.Chrome()

    def get_routes_from_page():
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        for route in soup.select('.route_details > a'):
            route_url = route['href']
            title = route['title']
            if not route_url.startswith(url):
                route_url = 'https://www.redbus.in'+ route_url
            route_details[title] = route_url

    driver.get(url)

    # Initial fetching of routes
    get_routes_from_page()

    try:
        pagination = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'DC_117_paginationTable'))
        )
        page_tabs = pagination.find_elements(By.CLASS_NAME, 'DC_117_pageTabs')
        page_numbers = [tab.text.strip() for tab in page_tabs if tab.text.strip().isdigit()]

        for page_num in page_numbers[1:]:
            tab = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f"//div[text()='{page_num}' and contains(@class, 'DC_117_pageTabs')]"))
            )
            actions = ActionChains(driver)
            actions.move_to_element(tab).perform()

            try:
                tab.click()
                time.sleep(2)  # Adjust sleep as necessary
            except Exception as e:
                print(f"Error clicking on page {page_num}: {e}")

            WebDriverWait(driver, 10).until(
                EC.text_to_be_present_in_element((By.CLASS_NAME, 'DC_117_pageActive'), page_num)
            )
            get_routes_from_page()
    except Exception as e:
        print(f"Pagination not found or error in pagination: {e}")
        print(driver.page_source)  # For debugging
    finally:
        driver.quit()

    return route_details

region_dict = {
'South': ['https://www.redbus.in/online-booking/tnstc',
  'https://www.redbus.in/online-booking/puducherry-road-transport-corporation-prtc',
  'https://www.redbus.in/online-booking/ktcl',
  'https://www.redbus.in/online-booking/ksrtc-kerala',
  'https://www.redbus.in/online-booking/apsrtc',
  'https://www.redbus.in/online-booking/tsrtc'],
 'East': ['https://www.redbus.in/online-booking/bihar-state-tourism-development-corporation',
  'https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc',
  'https://www.redbus.in/online-booking/west-bengal-transport-corporation',
  'https://www.redbus.in/online-booking/north-bengal-state-transport-corporation',
  'https://www.redbus.in/online-booking/bsrtc-operated-by-vip-travels',
  'https://www.redbus.in/online-booking/wbtc-ctc',
  'https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc'],
 'North': ['https://www.redbus.in/online-booking/pepsu',
  'https://www.redbus.in/online-booking/rsrtc',
  'https://www.redbus.in/online-booking/hrtc',
  'https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc',
  'https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu',
  'https://www.redbus.in/online-booking/pepsu-punjab',
  'https://www.redbus.in/online-booking/himachal-pradesh-tourism-development-corporation-hptdc'],
 'NorthEast': ['https://www.redbus.in/online-booking/sikkim-nationalised-transport-snt',
  'https://www.redbus.in/online-booking/meghalaya-transport-corporation-mtc',
  'https://www.redbus.in/online-booking/astc',
  'https://www.redbus.in/online-booking/kaac-transport'],
 'West': ['https://www.redbus.in/online-booking/gsrtc']
}

region_route_links = []
pagination_urls = []

for region, urls in region_dict.items():
    for url in urls:
        route_details = fetch_route_details(url)
    #     if route_details:
    #         pagination_urls.append(url)
    #     if len(pagination_urls) >= 10:
    #         break
    # if len(pagination_urls) >= 10:
    #     break

# for url in pagination_urls:
#     route_details = fetch_route_details(url)
        region_route_links.append(route_details)

print(region_route_links)


Pagination not found or error in pagination: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF66096EEB2+31554]
	(No symbol) [0x00007FF6608E7EE9]
	(No symbol) [0x00007FF6607A872A]
	(No symbol) [0x00007FF6607F8434]
	(No symbol) [0x00007FF6607F853C]
	(No symbol) [0x00007FF66083F6A7]
	(No symbol) [0x00007FF66081D06F]
	(No symbol) [0x00007FF66083C977]
	(No symbol) [0x00007FF66081CDD3]
	(No symbol) [0x00007FF6607EA33B]
	(No symbol) [0x00007FF6607EAED1]
	GetHandleVerifier [0x00007FF660C78B2D+3217341]
	GetHandleVerifier [0x00007FF660CC5AF3+3532675]
	GetHandleVerifier [0x00007FF660CBB0F0+3489152]
	GetHandleVerifier [0x00007FF660A1E786+750614]
	(No symbol) [0x00007FF6608F376F]
	(No symbol) [0x00007FF6608EEB24]
	(No symbol) [0x00007FF6608EECB2]
	(No symbol) [0x00007FF6608DE17F]
	BaseThreadInitThunk [0x00007FFD682D257D+29]
	RtlUserThreadStart [0x00007FFD6A2CAF28+40]

<html lang="en" class="wf-montserrat-n4-active wf-active"><head>
               <title>TNSTC Bus Ticket Booking online, Bus Reserv

In [4]:
region_route_links

[{},
 {},
 {'Pune to Goa': 'https://www.redbus.in/bus-tickets/pune-to-goa',
  'Goa to Pune': 'https://www.redbus.in/bus-tickets/goa-to-pune',
  'Mumbai to Goa': 'https://www.redbus.in/bus-tickets/mumbai-to-goa',
  'Goa to Mumbai': 'https://www.redbus.in/bus-tickets/goa-to-mumbai',
  'Pandharpur to Goa': 'https://www.redbus.in/bus-tickets/pandharpur-to-goa',
  'Bangalore to Goa': 'https://www.redbus.in/bus-tickets/bangalore-to-goa',
  'Goa to Pandharpur': 'https://www.redbus.in/bus-tickets/goa-to-pandharpur',
  'Belagavi to Goa': 'https://www.redbus.in/bus-tickets/belagavi-to-goa',
  'Goa to Bangalore': 'https://www.redbus.in/bus-tickets/goa-to-bangalore',
  'Solapur to Goa': 'https://www.redbus.in/bus-tickets/solapur-to-goa',
  'Goa to Kolhapur(Maharashtra)': 'https://www.redbus.in/bus-tickets/goa-to-kolhapur-maharashtra',
  'Goa to Solapur': 'https://www.redbus.in/bus-tickets/goa-to-solapur',
  'Goa to Sangola (Solapur)': 'https://www.redbus.in/bus-tickets/goa-to-sangola',
  'Sangola 

USING EACH ROUTE LINK : LAUNCH THE ROUTE PAGE AND CLICK ON RTC SECTION AND CLICK BUTTON VIEW BUSES AND THEN EXTRACT THE ROUTE NAME, AVIALBLE GOV BUS DETAILS LIKE BUS TYPE, FARE, SEAT AVAIALBILITY FROM ALL PAGES AND SAVE INFORM OF CSV FILE

In [1]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pprint

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Dictionary containing region-wise route URLs
region_route_links = [
 

 {'Diphu to Guwahati': 'https://www.redbus.in/bus-tickets/diphu-to-guwahati',
  'Guwahati to Diphu': 'https://www.redbus.in/bus-tickets/guwahati-to-diphu',
  'Bokolia (assam) to Guwahati': 'https://www.redbus.in/bus-tickets/bokolia-assam-to-guwahati',
  'Dokmoka to Guwahati': 'https://www.redbus.in/bus-tickets/dokmoka-to-guwahati',
  'Guwahati to Dokmoka': 'https://www.redbus.in/bus-tickets/guwahati-to-dokmoka',
  'Guwahati to Bokolia (assam)': 'https://www.redbus.in/bus-tickets/guwahati-to-bokolia-assam',
  'Langhin (assam) to Guwahati': 'https://www.redbus.in/bus-tickets/langhin-assam-to-guwahati',
  'Diphu to Hamren': 'https://www.redbus.in/bus-tickets/diphu-to-hamren',
  'Guwahati to Langhin (assam)': 'https://www.redbus.in/bus-tickets/guwahati-to-langhin-assam',
  'Guwahati to Manja (assam)': 'https://www.redbus.in/bus-tickets/guwahati-to-manja-assam',
  'Manja (assam) to Guwahati': 'https://www.redbus.in/bus-tickets/manja-assam-to-guwahati',
  'Hamren to Diphu': 'https://www.redbus.in/bus-tickets/hamren-to-diphu'}
]


# Initialize an empty dictionary to store all bus details
all_bus_details = {}

# Iterate through each region's routes
for region_routes in region_route_links:
    for route, url in region_routes.items():
        # Open the URL
        driver.get(url)

        # Wait for the page to load
        time.sleep(5)

        try:
            # Find all bus sections
            bus_sections = driver.find_elements(By.XPATH, "//div[contains(@class, 'f-bold grop-name')]")

            # Click on the relevant bus section
            for section in bus_sections:
                section_title = section.get_attribute("title")
                if section_title and 'RTC' in section_title:  # Check if the section title contains 'RTC'
                    section.click()
                    print(f"Clicked on {section_title} section for route: {route}")

                    # Click on "View Buses"
                    view_buses_button = WebDriverWait(driver, 20).until(
                        EC.element_to_be_clickable((By.XPATH, "//div[@class='button' and text()='View Buses']"))
                    )
                    view_buses_button.click()
                    print(f"Clicked on View Buses for route: {route}")

                    # Wait for the bus details to load
                    time.sleep(5)

                    # Scroll to the bottom to load all bus items
                    last_height = driver.execute_script("return document.body.scrollHeight")
                    while True:
                        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        time.sleep(3)
                        new_height = driver.execute_script("return document.body.scrollHeight")
                        if new_height == last_height:
                            break
                        last_height = new_height

                    # Extract bus details
                    bus_details = []

                    bus_items = driver.find_elements(By.CLASS_NAME, 'clearfix.bus-item-details')

                    for bus in bus_items:
                        try:
                            # Check if all required elements are present
                            bus_name_elem = bus.find_element(By.CLASS_NAME, 'travels.lh-24.f-bold.d-color')
                            bus_type_elem = bus.find_element(By.CLASS_NAME, 'bus-type.f-12.m-top-16.l-color.evBus')
                            departing_time_elem = bus.find_element(By.CLASS_NAME, 'dp-time.f-19.d-color.f-bold')
                            duration_elem = bus.find_element(By.CLASS_NAME, 'dur.l-color.lh-24')
                            reaching_time_elem = bus.find_element(By.CLASS_NAME, 'bp-time.f-19.d-color.disp-Inline')

                            star_rating_elem = None
                            try:
                                star_rating_elem = bus.find_element(By.CLASS_NAME, 'rating-sec.lh-24').find_element(By.TAG_NAME, 'span')
                            except:
                                star_rating_elem = None

                            seat_fare_elem = bus.find_element(By.CLASS_NAME, 'seat-fare').find_element(By.CLASS_NAME, 'f-19.f-bold')
                            seats_available_elem = bus.find_element(By.CLASS_NAME, 'column-eight.w-15.fl')

                            # Extract text from elements
                            bus_name = bus_name_elem.text if bus_name_elem else ''
                            bus_type = bus_type_elem.text if bus_type_elem else ''
                            departing_time = departing_time_elem.text if departing_time_elem else ''
                            duration = duration_elem.text if duration_elem else ''
                            reaching_time = reaching_time_elem.text if reaching_time_elem else ''
                            star_rating = star_rating_elem.text if star_rating_elem else 'N/A'
                            seat_fare = seat_fare_elem.text if seat_fare_elem else ''
                            seats_available = seats_available_elem.text if seats_available_elem else ''

                            # Append to bus_details list
                            bus_details.append({
                                'bus_name': bus_name,
                                'bus_type': bus_type,
                                'departing_time': departing_time,
                                'duration': duration,
                                'reaching_time': reaching_time,
                                'star_rating': star_rating,
                                'seat_fare': seat_fare,
                                'seats_available': seats_available
                            })
                        except Exception as e:
                            print(f"Error extracting bus details for route {route}: {e}")
                            continue

                    all_bus_details[route] = bus_details
                    break  # Break the loop after finding and clicking the relevant section

        except Exception as e:
            print(f"An error occurred for route {route}: {e}")
            continue

# Close the WebDriver
driver.quit()

# Save the extracted bus details to a CSV file
csv_file = 'bus_details.csv'
try:
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Route', 'Bus Name', 'Bus Type', 'Departing Time', 'Duration', 'Reaching Time', 'Star Rating', 'Seat Fare', 'Seats Available']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for route, details in all_bus_details.items():
            for detail in details:
                writer.writerow({
                    'Route': route,
                    'Bus Name': detail['bus_name'],
                    'Bus Type': detail['bus_type'],
                    'Departing Time': detail['departing_time'],
                    'Duration': detail['duration'],
                    'Reaching Time': detail['reaching_time'],
                    'Star Rating': detail['star_rating'],
                    'Seat Fare': detail['seat_fare'],
                    'Seats Available': detail['seats_available']
                })

    print(f"Bus details saved successfully to {csv_file}")

except IOError:
    print(f"Failed to write CSV file: {csv_file}")


Bus details saved successfully to bus_details.csv


FOR SOME ROUTE LINKS, removed the section where the code clicks on the "View Buses" button and instead directly attempts to extract bus details if available. LAUNCH THE ROUTE PAGE AND THEN EXTRACT THE ROUTE NAME, AVIALBLE GOV BUS DETAILS LIKE BUS TYPE, FARE, SEAT AVAIALBILITY FROM ALL PAGES AND SAVE INFORM OF CSV FILE 

In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Dictionary containing region-wise route URLs
region_route_links = [
    {'Guwahati to Shillong': 'https://www.redbus.in/bus-tickets/guwahati-to-shillong',
     'Shillong to Tura (Meghalaya)': 'https://www.redbus.in/bus-tickets/shillong-to-tura-meghalaya',
     'Shillong to Guwahati': 'https://www.redbus.in/bus-tickets/shillong-to-guwahati',
     'Tura (Meghalaya) to Shillong': 'https://www.redbus.in/bus-tickets/tura-meghalaya-to-shillong',
     'Shillong to Silchar': 'https://www.redbus.in/bus-tickets/shillong-to-silchar',
     'Silchar to Shillong': 'https://www.redbus.in/bus-tickets/silchar-to-shillong',
     'Shillong to Karimganj': 'https://www.redbus.in/bus-tickets/shillong-to-karimganj',
     'Shillong to Williamnagar (Meghalaya)': 'https://www.redbus.in/bus-tickets/shillong-to-williamnagar-meghalaya',
     'Karimganj to Shillong': 'https://www.redbus.in/bus-tickets/karimganj-to-shillong',
     'Williamnagar (Meghalaya) to Shillong': 'https://www.redbus.in/bus-tickets/williamnagar-meghalaya-to-shillong',
     'Shillong to Hailakandi': 'https://www.redbus.in/bus-tickets/shillong-to-hailakandi',
     'Shillong to Ramkrishnanagar': 'https://www.redbus.in/bus-tickets/shillong-to-ramkrishnanagar',
     'Hailakandi to Shillong': 'https://www.redbus.in/bus-tickets/hailakandi-to-shillong',
     'Ramkrishnanagar to Shillong': 'https://www.redbus.in/bus-tickets/ramkrishnanagar-to-shillong',
     'Shillong to Phulbari': 'https://www.redbus.in/bus-tickets/shillong-to-phulbari'},
    {'Dhekiajuli to Guwahati': 'https://www.redbus.in/bus-tickets/dhekiajuli-to-guwahati',
     'Jorhat to Dibrugarh': 'https://www.redbus.in/bus-tickets/jorhat-to-dibrugarh',
     'Jorhat to Dhemaji': 'https://www.redbus.in/bus-tickets/jorhat-to-dhemaji',
     'Sibsagar to North Lakhimpur': 'https://www.redbus.in/bus-tickets/sibsagar-to-north-lakhimpur',
     'Dhemaji to Jorhat': 'https://www.redbus.in/bus-tickets/dhemaji-to-jorhat',
     'Tezpur to Dibrugarh': 'https://www.redbus.in/bus-tickets/tezpur-to-dibrugarh',
     'Haflong to Guwahati': 'https://www.redbus.in/bus-tickets/haflong-to-guwahati',
     'North Lakhimpur to Dibrugarh': 'https://www.redbus.in/bus-tickets/north-lakhimpur-to-dibrugarh',
     'Jorhat to Tinsukia': 'https://www.redbus.in/bus-tickets/jorhat-to-tinsukia',
     'Dibrugarh to Tezpur': 'https://www.redbus.in/bus-tickets/dibrugarh-to-tezpur',
     'Guwahati to Biswanath Charali': 'https://www.redbus.in/bus-tickets/guwahati-to-biswanath-charali',
     'Guwahati to Haflong': 'https://www.redbus.in/bus-tickets/guwahati-to-haflong',
     'Dibrugarh to North Lakhimpur': 'https://www.redbus.in/bus-tickets/dibrugarh-to-north-lakhimpur',
     'Nagaon (Assam) to Haflong': 'https://www.redbus.in/bus-tickets/nagaon-to-haflong',
     'Tezpur to Moran': 'https://www.redbus.in/bus-tickets/tezpur-to-moran',
     'Dibrugarh to Jorhat': 'https://www.redbus.in/bus-tickets/dibrugarh-to-jorhat',
     'Guwahati to Silchar': 'https://www.redbus.in/bus-tickets/guwahati-to-silchar',
     'Bihpuria to Dibrugarh': 'https://www.redbus.in/bus-tickets/bihpuria-to-dibrugarh',
     'Haflong to Nagaon (Assam)': 'https://www.redbus.in/bus-tickets/haflong-to-nagaon',
     'North Lakhimpur to Tezpur': 'https://www.redbus.in/bus-tickets/north-lakhimpur-to-tezpur',
     'Biswanath Charali to Guwahati': 'https://www.redbus.in/bus-tickets/biswanath-charali-to-guwahati',
     'Biswanath Charali to Dibrugarh': 'https://www.redbus.in/bus-tickets/biswanath-charali-to-dibrugarh',
     'Tinsukia to Jorhat': 'https://www.redbus.in/bus-tickets/tinsukia-to-jorhat',
     'Moran to Tezpur': 'https://www.redbus.in/bus-tickets/moran-to-tezpur',
     'Jorhat to Gogamukh': 'https://www.redbus.in/bus-tickets/jorhat-to-gogamukh',
     'Dibrugarh to Biswanath Charali': 'https://www.redbus.in/bus-tickets/dibrugarh-to-biswanath-charali',
     'Gohpur to Guwahati': 'https://www.redbus.in/bus-tickets/gohpur-to-guwahati',
     'Tinsukia to Tezpur': 'https://www.redbus.in/bus-tickets/tinsukia-to-tezpur',
     'North Lakhimpur to Golaghat': 'https://www.redbus.in/bus-tickets/north-lakhimpur-to-golaghat',
     'Golaghat to North Lakhimpur': 'https://www.redbus.in/bus-tickets/golaghat-to-north-lakhimpur',
     'Silchar to Guwahati': 'https://www.redbus.in/bus-tickets/silchar-to-guwahati',
     'Bokakhat to Dibrugarh': 'https://www.redbus.in/bus-tickets/bokakhat-to-dibrugarh',
     'North Lakhimpur to Moran': 'https://www.redbus.in/bus-tickets/north-lakhimpur-to-moran',
     'Tezpur to North Lakhimpur': 'https://www.redbus.in/bus-tickets/tezpur-to-north-lakhimpur',
     'Tezpur to Tinsukia': 'https://www.redbus.in/bus-tickets/tezpur-to-tinsukia',
     'Dibrugarh to Bihpuria': 'https://www.redbus.in/bus-tickets/dibrugarh-to-bihpuria',
     'Dibrugarh to Bokakhat': 'https://www.redbus.in/bus-tickets/dibrugarh-to-bokakhat'},
]

# Initialize an empty dictionary to store all bus details
all_bus_details = {}

# Iterate through each region's routes
for region_routes in region_route_links:
    for route, url in region_routes.items():
        # Open the URL
        driver.get(url)

        # Wait for the page to load
        time.sleep(5)

        try:
            # Find all bus sections
            bus_sections = driver.find_elements(By.XPATH, "//div[contains(@class, 'f-bold grop-name')]")

            # Click on the relevant bus section
            for section in bus_sections:
                section_title = section.get_attribute("title")
                if section_title and 'RTC' in section_title:  # Check if the section title contains 'RTC'
                    section.click()
                    print(f"Clicked on {section_title} section for route: {route}")

                    # Wait for the bus details to load
                    time.sleep(5)

                    # Scroll to the bottom to load all bus items
                    last_height = driver.execute_script("return document.body.scrollHeight")
                    while True:
                        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        time.sleep(3)
                        new_height = driver.execute_script("return document.body.scrollHeight")
                        if new_height == last_height:
                            break
                        last_height = new_height

                    # Extract bus details
                    bus_details = []

                    bus_items = driver.find_elements(By.CLASS_NAME, 'clearfix.bus-item-details')

                    if not bus_items:
                        print(f"No buses available for route {route}. Skipping to next route.")
                        break

                    for bus in bus_items:
                        try:
                            # Check if all required elements are present
                            bus_name_elem = bus.find_element(By.CLASS_NAME, 'travels.lh-24.f-bold.d-color')
                            bus_type_elem = bus.find_element(By.CLASS_NAME, 'bus-type.f-12.m-top-16.l-color.evBus')
                            departing_time_elem = bus.find_element(By.CLASS_NAME, 'dp-time.f-19.d-color.f-bold')
                            duration_elem = bus.find_element(By.CLASS_NAME, 'dur.l-color.lh-24')
                            reaching_time_elem = bus.find_element(By.CLASS_NAME, 'bp-time.f-19.d-color.disp-Inline')

                            star_rating_elem = None
                            try:
                                star_rating_elem = bus.find_element(By.CLASS_NAME, 'rating-sec.lh-24').find_element(By.TAG_NAME, 'span')
                            except:
                                star_rating_elem = None

                            seat_fare_elem = bus.find_element(By.CLASS_NAME, 'seat-fare').find_element(By.CLASS_NAME, 'f-19.f-bold')
                            seats_available_elem = bus.find_element(By.CLASS_NAME, 'column-eight.w-15.fl')

                            # Extract text from elements
                            bus_name = bus_name_elem.text if bus_name_elem else ''
                            bus_type = bus_type_elem.text if bus_type_elem else ''
                            departing_time = departing_time_elem.text if departing_time_elem else ''
                            duration = duration_elem.text if duration_elem else ''
                            reaching_time = reaching_time_elem.text if reaching_time_elem else ''
                            star_rating = star_rating_elem.text if star_rating_elem else 'N/A'
                            seat_fare = seat_fare_elem.text if seat_fare_elem else ''
                            seats_available = seats_available_elem.text if seats_available_elem else ''

                            # Append to bus_details list
                            bus_details.append({
                                'bus_name': bus_name,
                                'bus_type': bus_type,
                                'departing_time': departing_time,
                                'duration': duration,
                                'reaching_time': reaching_time,
                                'star_rating': star_rating,
                                'seat_fare': seat_fare,
                                'seats_available': seats_available
                            })
                        except Exception as e:
                            print(f"Error extracting bus details for route {route}: {e}")
                            continue

                    all_bus_details[route] = bus_details
                    break  # Break the loop after finding and clicking the relevant section

        except Exception as e:
            print(f"An error occurred for route {route}: {e}")
            continue

# Close the WebDriver
driver.quit()

# Save the extracted bus details to a CSV file
csv_file = 'bus_details.csv'
try:
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Route', 'Bus Name', 'Bus Type', 'Departing Time', 'Duration', 'Reaching Time', 'Star Rating', 'Seat Fare', 'Seats Available']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for route, details in all_bus_details.items():
            for detail in details:
                writer.writerow({
                    'Route': route,
                    'Bus Name': detail['bus_name'],
                    'Bus Type': detail['bus_type'],
                    'Departing Time': detail['departing_time'],
                    'Duration': detail['duration'],
                    'Reaching Time': detail['reaching_time'],
                    'Star Rating': detail['star_rating'],
                    'Seat Fare': detail['seat_fare'],
                    'Seats Available': detail['seats_available']
                })

    print(f"Bus details saved successfully to {csv_file}")

except IOError:
    print(f"Failed to write CSV file: {csv_file}")
